## Load dataset


In [12]:
import torch
import pandas as pd
from hmc.dataset import HMCDataset
from torch.utils.data import Dataset, DataLoader

In [2]:
from hmc.utils import __load_json__

In [3]:
import os
import json

In [4]:
base_path = "/mnt/disks/data/fma/trains"
id = "hierarchical_tworoots"

train_path = os.path.join(base_path,id)
metadata_path = os.path.join(train_path,'metadata.json')
labels_path = os.path.join(train_path,'labels.json')
torch_path =os.path.join(train_path,'torch')

In [5]:
metadata = __load_json__(metadata_path)

In [6]:
labels = __load_json__(labels_path)

In [7]:
metadata['max_depth']

4

In [8]:
#df_val = pd.read_csv(metadata['val_csv'])

In [9]:
#df_val

In [ ]:
BUFFER_SIZE = 10

class HMCDataset(Dataset):
    def __init__(self, files,levels_size):
        self.files = files
        self.levels_size = levels_size
        self.data = self.load_data()

    def load_data(self):
        # Carregar dados a partir dos arquivos e retornar uma lista de exemplos
        data = []
        for file in os.listdir(self.files):
            file_path = os.path.join(self.files, file)
            data.extend(torch.load(file_path))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        features = torch.tensor(example['features'], dtype=torch.float32)
        #track_id = example['track_id']
        #labels = {f'level{level}': torch.tensor(example[f'level{level}'], dtype=torch.float32)
        #          for level in range(1, len(self.levels_size) + 1)}
        labels = [torch.tensor(np.array(example[f'level{level}']), dtype=torch.float32)
                  for level in range(1, len(self.levels_size) + 1)]
        return  features, labels


In [10]:
metadata['levels_size']

[2, 29, 15, 2]

In [22]:
import sys
import types

# Create a dummy dataset module with dataset_torch submodule
dataset = types.ModuleType('dataset')
dataset_torch = types.ModuleType('dataset_torch')
dataset.dataset_torch = dataset_torch
sys.modules['dataset'] = dataset
sys.modules['dataset.dataset_torch'] = dataset_torch


dataset_torch.MusicDataset = HMCDataset
sys.modules['dataset_torch'] = dataset_torch


# Carregar o dataset salvo
loaded_dataset = HMCDataset(metadata['train_torch_path'], levels_size=metadata['levels_size'])

# Criação do DataLoader
ds_validation = DataLoader(loaded_dataset, batch_size=64, shuffle=True)


In [23]:
len(loaded_dataset)

18209

In [25]:
for inputs, labels in ds_validation:
    print(labels[0])
    break

tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1

In [18]:
#ds_train = Dataset(trainset_pattern, 10, 32, depth=metadata['max_depth']).build(df=True)

In [28]:
val_dataset_array = next(iter(ds_validation))[0].numpy()

In [29]:
val_dataset_array

array([[-0.05795394, -0.03638636,  0.22502355, ...,  0.01868322,
         0.16242169,  0.0790742 ],
       [-0.01704843,  0.01406341,  0.00475705, ..., -0.00316535,
         0.19925644,  0.01645163],
       [-0.01152799, -0.05991222, -0.03120563, ...,  0.0007601 ,
        -0.04211006, -0.02922575],
       ...,
       [ 0.00217863,  0.01848902, -0.03141269, ...,  0.00428359,
         0.07604198, -0.00207548],
       [ 0.50329274,  0.05604462,  0.04162657, ..., -0.01690188,
        -0.01881945, -0.04627268],
       [ 0.05001318, -0.03526527,  0.02279306, ...,  0.01409413,
        -0.01773393,  0.00630476]], dtype=float32)